In [1]:
pip install google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google.oauth2.service_account import Credentials
import io

# Ruta al archivo de credenciales JSON
SERVICE_ACCOUNT_FILE = 'C:/Users/prorcle/Desktop/Riccardo/key/credenciales.json'  # Cambia por la ubicación de tu archivo

# Configurar los permisos de la API
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Construir el servicio de la API de Google Drive
service = build('drive', 'v3', credentials=creds)

# ID de la carpeta en Google Drive
folder_id = '17a0kvZD1ZfRPyMCgrNlr4vCCdqN9djCv'  # Cambia esto por el ID de tu carpeta

# Listar los archivos en la carpeta
results = service.files().list(
    q=f"'{folder_id}' in parents",
    fields="files(id, name)").execute()

files = results.get('files', [])
if not files:
    print('No se encontraron archivos en la carpeta.')
else:
    print('Archivos encontrados:')
    for file in files:
        print(f"Descargando: {file['name']}")

        # Descargar el archivo
        request = service.files().get_media(fileId=file['id'])
        file_path = f"./{file['name']}"  # Cambia la ruta si quieres guardar en otro lugar
        with io.FileIO(file_path, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
                print(f"Progreso de descarga: {int(status.progress() * 100)}%")


Archivos encontrados:
Descargando: tickets_prueba_v2.txt
Progreso de descarga: 16%
Progreso de descarga: 32%
Progreso de descarga: 49%
Progreso de descarga: 65%
Progreso de descarga: 81%
Progreso de descarga: 98%
Progreso de descarga: 100%
Descargando: productos_prueba_v2.txt
Progreso de descarga: 100%


In [2]:
pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import psycopg2

# ----------------------------
# Cargar el archivo en pandas
# ----------------------------
file_tickets = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/tickets_prueba_v2.txt"

# Leer el archivo TXT con coma como delimitador
df_tickets = pd.read_csv(file_tickets, delimiter=";", low_memory=False)
print("Datos cargados en pandas:")
print(df_tickets.head())

# ----------------------------
# Conexión a PostgreSQL
# ----------------------------
try:
    conn = psycopg2.connect(
        host="127.0.0.1",
        database="ingdatos",  # Cambia al nombre de tu base de datos
        user="postgres",
        password="123456",  # Cambia por tu contraseña
        port="5432"
    )
    cursor = conn.cursor()
    print("Conexión a PostgreSQL establecida.")
except Exception as e:
    print(f"Error al conectar a PostgreSQL: {e}")
    exit()

# ----------------------------
# Crear la tabla en PostgreSQL
# ----------------------------
table_name = "tickets"  # Cambia el nombre de la tabla según corresponda

# Crear la tabla basada en las columnas del DataFrame
columns = ", ".join([f'"{col}" TEXT' for col in df_tickets.columns])  # Cambia "TEXT" si necesitas tipos específicos
create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns});'

try:
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Tabla '{table_name}' creada correctamente.")
except Exception as e:
    print(f"Error al crear la tabla '{table_name}': {e}")
    cursor.close()
    conn.close()
    exit()

# ----------------------------
# Insertar los datos en la tabla
# ----------------------------
# Convertir el DataFrame a una lista de tuplas
data_tuples = [tuple(row) for row in df_tickets.to_numpy()]

# Crear la consulta INSERT
cols = ', '.join([f'"{col}"' for col in df_tickets.columns])
placeholders = ', '.join(['%s'] * len(df_tickets.columns))
insert_query = f'INSERT INTO {table_name} ({cols}) VALUES ({placeholders});'

try:
    cursor.executemany(insert_query, data_tuples)
    conn.commit()
    print(f"{len(data_tuples)} registros insertados en la tabla '{table_name}'.")
except Exception as e:
    print(f"Error al insertar datos: {e}")

# ----------------------------
# Cerrar la conexión
# ----------------------------
cursor.close()
conn.close()
print("Conexión a PostgreSQL cerrada.")

Datos cargados en pandas:
   punto ticket       fecha      hora  eancode  \
0      1  18220  2023-06-01  05:06:00     5400   
1      1  18220  2023-06-01  05:06:00     5414   
2      1  18220  2023-06-01  05:06:00   711951   
3      1  18221  2023-06-01  05:11:00       37   
4      1  18221  2023-06-01  05:11:00      545   

                                     ean_desc  unidades_vendidas  \
0                            *PAN FELIPE X KG              1.025   
1                        *GALLETA MOLIDA X KG              1.925   
2  BOLSA CAMISILLA 50x60 LJ VERDE CERTIFICADA              1.000   
3                  CHORIZO C/QUESO OCHSI x KG              0.290   
4               HARINA DE MAIZ x KG GRANOPAR.              1.010   

   precio_regular  precio_promocional tipo_venta  idcadena  \
0          6850.0              6850.0          P         5   
1          6600.0              6600.0          P         5   
2           200.0               200.0          P         5   
3         44500.

In [6]:
pip install charset-normalizer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from charset_normalizer import detect

file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2.txt"

# Detectar la codificación
with open(file_productos, "rb") as file:
    result = detect(file.read())
    encoding_detected = result['encoding']
    print(f"Codificación detectada: {encoding_detected}")

# Usar la codificación detectada para leer el archivo
with open(file_productos, "r", encoding=encoding_detected, errors="replace") as file:
    for i, line in enumerate(file):
        if i >= 1675 and i <= 1678:  # Línea problemática y alrededores
            print(f"Línea {i + 1}: {line}")


Codificación detectada: ISO-8859-2
Línea 1676: 5;1315;ENTRECOSTILLA X KG ;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;151;Carne Vacuna;CABAĂA DON VICENTE S.R.L.;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829922;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1677: 5;1316;COSTILLA VACUNA PRECOCIDA 18HS X KG;;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;153;Carne Vacuna Terceros;AGROPECUARIA  ITABO S.A.C.;EL PATRON;None;None;KL;2024-01-16 15:29:19.391424;1829923;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1678: 5;1317;*CHORIZO MISIONERO GRANEL C/YS/ PICANTE XKG SANTIN;2;Perecedero (Frescos);9;FiambrerĂ­a;386;Autoservicios;1125;Autoservicios Embutidos;LUCIANO CONCEPCION RUIZ DIAZ MASCAREĂO;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829924;PERECEDEROS;EMBUTIDOS;CHORIZO;None

Línea 1679: 5;1318;PLATO DE PICADITO DE CARNE x UNID;23;Alimentos Prod. TC;80;ProducciĂłn;303;Minuta;954;Otras Minutas;INNOVACIONES ALIMENTARIAS S.R.L;NO DEFIN

In [9]:
import pandas as pd
# Leer el archivo TXT con coma como delimitador
df_productos = pd.read_csv(file_productos, delimiter=";", low_memory=False)
print("Datos cargados en pandas:")
print(df_productos.head())

ParserError: Error tokenizing data. C error: Expected 22 fields in line 1677, saw 23


In [10]:
with open(file_productos, "r", encoding="utf-8", errors="replace") as file:
    for i, line in enumerate(file):
        if i >= 1675 and i <= 1678:  # Inspeccionar líneas relevantes
            print(f"Línea {i + 1}: {line}")

Línea 1676: 5;1315;ENTRECOSTILLA X KG ;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;151;Carne Vacuna;CABAÑA DON VICENTE S.R.L.;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829922;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1677: 5;1316;COSTILLA VACUNA PRECOCIDA 18HS X KG;;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;153;Carne Vacuna Terceros;AGROPECUARIA  ITABO S.A.C.;EL PATRON;None;None;KL;2024-01-16 15:29:19.391424;1829923;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1678: 5;1317;*CHORIZO MISIONERO GRANEL C/YS/ PICANTE XKG SANTIN;2;Perecedero (Frescos);9;Fiambrería;386;Autoservicios;1125;Autoservicios Embutidos;LUCIANO CONCEPCION RUIZ DIAZ MASCAREÑO;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829924;PERECEDEROS;EMBUTIDOS;CHORIZO;None

Línea 1679: 5;1318;PLATO DE PICADITO DE CARNE x UNID;23;Alimentos Prod. TC;80;Producción;303;Minuta;954;Otras Minutas;INNOVACIONES ALIMENTARIAS S.R.L;NO DEFINIDA;None;None;UNID;2024-01-16 15:29:19.

In [11]:
file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2.txt"
clean_file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_cleaned.txt"

# Abrir el archivo original y crear un archivo limpio
with open(file_productos, "r", encoding="utf-8", errors="replace") as infile, open(clean_file_productos, "w", encoding="utf-8") as outfile:
    for i, line in enumerate(infile):
        if i == 1676:  # Línea problemática (ajusta el índice según sea necesario)
            # Reemplazar delimitadores dobles
            line = line.replace(";;", ";")
        outfile.write(line)

print(f"Archivo limpio creado: {clean_file_productos}")

Archivo limpio creado: C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_cleaned.txt


In [12]:
with open(clean_file_productos, "r", encoding="utf-8", errors="replace") as file:
    for i, line in enumerate(file):
        if i >= 1675 and i <= 1678:  # Inspeccionar líneas relevantes
            print(f"Línea {i + 1}: {line}")

Línea 1676: 5;1315;ENTRECOSTILLA X KG ;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;151;Carne Vacuna;CABAÑA DON VICENTE S.R.L.;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829922;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1677: 5;1316;COSTILLA VACUNA PRECOCIDA 18HS X KG;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;153;Carne Vacuna Terceros;AGROPECUARIA  ITABO S.A.C.;EL PATRON;None;None;KL;2024-01-16 15:29:19.391424;1829923;PERECEDEROS;CARNICERIA;VACUNA;None

Línea 1678: 5;1317;*CHORIZO MISIONERO GRANEL C/YS/ PICANTE XKG SANTIN;2;Perecedero (Frescos);9;Fiambrería;386;Autoservicios;1125;Autoservicios Embutidos;LUCIANO CONCEPCION RUIZ DIAZ MASCAREÑO;NO DEFINIDA;None;None;KL;2024-01-16 15:29:19.391424;1829924;PERECEDEROS;EMBUTIDOS;CHORIZO;None

Línea 1679: 5;1318;PLATO DE PICADITO DE CARNE x UNID;23;Alimentos Prod. TC;80;Producción;303;Minuta;954;Otras Minutas;INNOVACIONES ALIMENTARIAS S.R.L;NO DEFINIDA;None;None;UNID;2024-01-16 15:29:19.3

In [13]:
import pandas as pd
# Leer el archivo TXT con coma como delimitador
df_productos = pd.read_csv(clean_file_productos, delimiter=";", low_memory=False)
print("Datos cargados en pandas:")
print(df_productos.head())

ParserError: Error tokenizing data. C error: Expected 22 fields in line 6094, saw 23


In [17]:
## Existen lineas que no cumplen con las cantidades de columnas
## Buscaremos identificar el universo de los datos sin problemas y con problemas

In [15]:
import pandas as pd

file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2.txt"
valid_lines_file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_valid.csv"
error_lines_file_productos = "C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_errors.txt"

valid_lines = []  # Lista para almacenar las líneas válidas
error_lines = []  # Lista para almacenar las líneas con errores

# Abrir el archivo y procesarlo línea por línea
with open(file_productos, "r", encoding="utf-8", errors="replace") as file:
    for i, line in enumerate(file):
        try:
            # Intentar leer cada línea como si fuera un DataFrame
            df = pd.DataFrame([line.strip().split(";")])  # Dividir por delimitador ";"
            
            # Si la línea tiene el número correcto de columnas, añadirla a las válidas
            if len(df.columns) == 22:  # Cambia 22 por el número esperado de columnas
                valid_lines.append(line)
            else:
                raise ValueError("Número incorrecto de columnas")  # Forzar excepción
        except Exception as e:
            # Si hay algún error, registrar la línea problemática
            error_lines.append((i + 1, line.strip(), str(e)))

# Guardar las líneas válidas en un archivo CSV
with open(valid_lines_file_productos, "w", encoding="utf-8") as valid_file:
    valid_file.writelines(valid_lines)

# Guardar las líneas problemáticas en un archivo de texto
with open(error_lines_file_productos, "w", encoding="utf-8") as error_file:
    for error in error_lines:
        error_file.write(f"Línea {error[0]}: {error[1]} - Error: {error[2]}\n")

print(f"Líneas válidas guardadas en: {valid_lines_file_productos}")
print(f"Líneas problemáticas guardadas en: {error_lines_file_productos}")


Líneas válidas guardadas en: C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_valid.csv
Líneas problemáticas guardadas en: C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_errors.txt


In [16]:
# Leer el archivo TXT con coma como delimitador
df_productos_clean = pd.read_csv(valid_lines_file_productos, delimiter=";", low_memory=False)
print("Datos cargados en pandas:")
print(df_productos_clean.head())

Datos cargados en pandas:
   idcadena       eancode                            descripcion  id_sector  \
0         5          3734              MORCILLA ROSCA FRANZ X KG          2   
1         5          3756   *MORCILLA GUARANI ENV AL VACIO X KG.          2   
2         5          3412                        ALBONDIGAS X KG          2   
3         5  341180480139  TAZA C/ PLATO 8x6,14 CM REF.726-48013          9   
4         5          3413                 MANI CON MIEL x UNIDAD          2   

                 sector  id_seccion                         seccion  \
0  Perecedero (Frescos)           9                      Fiambrería   
1  Perecedero (Frescos)           9                      Fiambrería   
2  Perecedero (Frescos)           8     Carniceria/ Avicola / Pesc.   
3                 Hogar          29                          Cocina   
4  Perecedero (Frescos)          12  Panaderia Pastelería y rotiser   

   id_categoria                       categoria  id_subcategoria  ...  \

In [17]:
# Mostrar estadísticas
total_lines = len(valid_lines) + len(error_lines)
print(f"Total de líneas en el archivo: {total_lines}")
print(f"Líneas válidas: {len(valid_lines)}")
print(f"Líneas problemáticas: {len(error_lines)}")

print(f"Líneas válidas guardadas en: {valid_lines_file_productos}")
print(f"Líneas problemáticas guardadas en: {error_lines_file_productos}")

Total de líneas en el archivo: 382270
Líneas válidas: 377241
Líneas problemáticas: 5029
Líneas válidas guardadas en: C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_valid.csv
Líneas problemáticas guardadas en: C:/Users/prorcle/Desktop/Datascience Proyects/IngData/productos_prueba_v2_errors.txt


In [16]:
# Mostrar específicamente qué líneas tienen problemas
print("\nLíneas problemáticas detectadas:")
for error in error_lines:
    print(f"Línea {error[0]}: {error[1]} - Error: {error[2]}")


Líneas problemáticas detectadas:
Línea 933: 5;488;QUESO ROQUEFORT DE OVEJA SOCIETE x KL;22;Delicatessen;68;Perecederos(Deli)_x000D_ - Error: Número incorrecto de columnas
Línea 934: _x000D_ - Error: Número incorrecto de columnas
Línea 935: ;231;Lacteos(Deli)_x000D_ - Error: Número incorrecto de columnas
Línea 936: ;798;Quesos(Deli)_x000D_ - Error: Número incorrecto de columnas
Línea 937: ;PARMALAT PARAGUAY S.A.;PRESIDENT;None;None;KL;2024-01-16 15:29:19.391424;1829110;PERECEDEROS;LACTEOS;QUESO;None - Error: Número incorrecto de columnas
Línea 1677: 5;1316;COSTILLA VACUNA PRECOCIDA 18HS X KG;;2;Perecedero (Frescos);8;Carniceria/ Avicola / Pesc.;41;Carniceria;153;Carne Vacuna Terceros;AGROPECUARIA  ITABO S.A.C.;EL PATRON;None;None;KL;2024-01-16 15:29:19.391424;1829923;PERECEDEROS;CARNICERIA;VACUNA;None - Error: Número incorrecto de columnas
Línea 4889: 5;13048;TOMATE SECO SIN ACEITE CABAÑA DE MARIA x KG;22;Delicatessen;67;Abarrotes(Deli)_x000D_ - Error: Número incorrecto de columnas
Lín

In [18]:

# ----------------------------
# Conexión a PostgreSQL
# ----------------------------
try:
    conn = psycopg2.connect(
        host="127.0.0.1",
        database="ingdatos",  # Cambia al nombre de tu base de datos
        user="postgres",
        password="123456",  # Cambia por tu contraseña
        port="5432"
    )
    cursor = conn.cursor()
    print("Conexión a PostgreSQL establecida.")
except Exception as e:
    print(f"Error al conectar a PostgreSQL: {e}")
    exit()

# ----------------------------
# Crear la tabla en PostgreSQL
# ----------------------------
table_name = "productos"  # Cambia el nombre de la tabla según corresponda

# Crear la tabla basada en las columnas del DataFrame
columns = ", ".join([f'"{col}" TEXT' for col in df_productos_clean.columns])  # Cambia "TEXT" si necesitas tipos específicos
create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns});'

try:
    cursor.execute(create_table_query)
    conn.commit()
    print(f"Tabla '{table_name}' creada correctamente.")
except Exception as e:
    print(f"Error al crear la tabla '{table_name}': {e}")
    cursor.close()
    conn.close()
    exit()

# ----------------------------
# Insertar los datos en la tabla
# ----------------------------
# Convertir el DataFrame a una lista de tuplas
data_tuples = [tuple(row) for row in df_productos_clean.to_numpy()]

# Crear la consulta INSERT
cols = ', '.join([f'"{col}"' for col in df_productos_clean.columns])
placeholders = ', '.join(['%s'] * len(df_productos_clean.columns))
insert_query = f'INSERT INTO {table_name} ({cols}) VALUES ({placeholders});'

try:
    cursor.executemany(insert_query, data_tuples)
    conn.commit()
    print(f"{len(data_tuples)} registros insertados en la tabla '{table_name}'.")
except Exception as e:
    print(f"Error al insertar datos: {e}")

# ----------------------------
# Cerrar la conexión
# ----------------------------
cursor.close()
conn.close()
print("Conexión a PostgreSQL cerrada.")

Conexión a PostgreSQL establecida.
Tabla 'productos' creada correctamente.
377240 registros insertados en la tabla 'productos'.
Conexión a PostgreSQL cerrada.
